## Importing Packages, loading data and textfunction

In [315]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import re
import time
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import kshingle as ks

In [106]:
with open("TVs-all-merged.json") as jsonFile:
    data = json.load(jsonFile)    

In [107]:
##-- Adding data to a list and splitting the modelID column from the rest
file = []
for i in range(len(list(data.values()))):
    if len(list(data.values())[i]) ==1:
        file.append(list(data.values())[i][0])
    else:
        for j in range(len(list(data.values())[i])):
            file.append(list(data.values())[i][1:][0])   
df = pd.DataFrame(file)
# display(df)
x_data = (df[['shop', 'url', 'featuresMap', 'title']])
y_data = (df['modelID'])

'29PFL4508/F7'

In [108]:
#make dictionary of all real duplicates
y_data2 = pd.DataFrame(y_data)
x_data2 = pd.DataFrame(x_data)

# modelIDList = textfunc2(str(list(y_data2['modelID'])))
modelIDList = ((list(y_data2['modelID'])))
modelTitleList = (list(x_data2['title']))

newListModelID = []
for i in modelIDList:
    i = i.replace("-", "")
    i = i.replace("/", "")
    newListModelID.append(i) #capital letters, no -, /
    
newListTitle = []
for i in modelTitleList:
    i = i.replace("-", "")
    i = i.replace("/", "")
    newListTitle.append(i)

In [109]:
titles2 = {}
for key in newListTitle:
#     print(key)
    for value in newListModelID:
#         print(value)
        titles2[key] = value
        newListModelID.remove(value)
        break

In [110]:

# print(titles2)
actualDupl = pd.DataFrame(np.zeros((len(titles2), len(titles2))), columns=titles2, index=titles2)

# display(actualDupl)

count=0
for key1 in actualDupl:
    for key2 in actualDupl:
        if key1 != key2: 
            if titles2[key1] == titles2[key2]:
                actualDupl.loc[key1, key2] = 1
                count+=1
print(count)
keys = actualDupl.keys()
# actualDupl.looc[key[9]]
# print(actualDupl.iloc[9,9])
keys[9]

64


'Newegg.com  Refurbished: Samsung 46" Class (45.9" Diag.) 1080p 240Hz LED HDTV UN46ES6580'

## Functions for removing inconsistencies and creating randomnumbers for minhashing

In [223]:
##-- Find all duplicate products using their modelID
y_data2 = pd.DataFrame(y_train)
dupCountList = []
realDupl=0
for i in y_data2['modelID']:
    if i not in dupCountList:
        dupCountList.append(i)
    else:
        realDupl+=1
realDupl


159

In [148]:
# ##-- Split x_data and y_data into x&y train & test data
# x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.37)


In [318]:
##--Bootstrap
numBootstrap = 5
sizeBootstrap = int(len(df) * 0.63)


for i in range(numBootstrap):
    bootdata = resample(df, n_samples = sizeBootstrap)
    x_boot = (bootdata[['shop', 'url', 'featuresMap', 'title']])
    y_boot = (bootdata['modelID'])
    fulltransformation(x_boot, y_boot, 500, 10, 50, 0.85)


Ended dataprocessing ( 1 / 12 ) 0.4332890510559082
Ended inputmatrix computation ( 2 / 12 ) 7.388461112976074
Ended minhashing computation ( 3 / 12 ) 7.396062135696411
Ended signaturematrix computation ( 4 / 12 ) 112.5577940940857
Ended splitting signaturematrix into bands ( 5 / 12 ) 112.65402221679688
Ended creating hashvalues for bands ( 6 / 12 ) 113.21923518180847
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 113.24145793914795
Ended computing all bucketpairs across bands ( 8 / 12 ) 113.24296498298645
Df 519
Ended LSH performance ( 9 / 12 ) 113.25141501426697
Ended computing jaccard and cosine similarities ( 10 / 12 ) 113.64292407035828
perfLSH [0.9885714285714285, 0.7510853835021708, 0.8536184210526316, 0.954992586021436, 0.00039836828351074]
perfSim [0.9885714285714285, 0.7510853835021708, 0.8536184210526316, 0.954992586021436, 0.00039836828351074]
Ended dataprocessing ( 1 / 12 ) 0.4012930393218994
Ended inputmatrix computation ( 2 / 12 ) 7.81688380241394
Ended m

In [151]:
##-- Function to remove inconsistencies in data
def textfunc2(location):
    data = re.sub(r'[^\w\s]','', location)
    items = data.lower()    
    items = items.split()    
    return items
textfunc2(y_train.iloc[0])

['un60d8000']

In [152]:
##-- Random number generator function with integer as input
def randomNumberGenerator(value):
    #create list to store random values such that random values are all created once when function is called
    randomNumbers = []
    indexValue = random.randint(0, len(allKeyWords) - 1)
    randomNumbers.append(indexValue)
    
    while value > 0:
        while indexValue in randomNumbers:
            indexValue = random.randint(0, len(allKeyWords) - 1)
        randomNumbers.append(indexValue)
        value=value-1
    return randomNumbers
p = 1543

## Function to process and perform LSH and classification on data

In [306]:
##-- Function to process and perform LSH and classification on data
def fulltransformation(x_train, y_train, hashNumber, b, r, thresholdSim):
    time0 = time.time()
    x_train2 = pd.DataFrame(x_train)
    y_train2 = pd.DataFrame(y_train)
        
    t = (1/b) ** (1/r)

    
    timer=1
    timerfin=12

##-- Data transformations
    listIDs = []
    for i in range(len(y_train)):
        listIDs.append(textfunc2(y_train.iloc[i]))

    for i in range(len(x_train2)):
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace('"', 'inch')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace('\"', 'inch')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace('inches', 'inch')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace('-inch', 'inch')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace(' inch', 'inch')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace(' hz', 'hz')
        x_train2['title'].iloc[i] = x_train2['title'].iloc[i].replace('-', '')

    
##-- Add all titles in string format to a list        
    titles_string=[]
    for i in range(len(x_train2)):
        titles_string.append(textfunc2(x_train2['title'].iloc[i]))  

##-- Add all individual lists of strings together in one list (callable by product index)
    totalWords = []
    for i in range(len(x_train2)):
        totalWords.append(titles_string[i])

##-- Remove all double keywords from totalWords (resulting list consists of only unique keywords)
    allKeyWords = []
    for i in range(len(x_train2)):
        for j in range(len(totalWords[i])):
            if totalWords[i][j] not in allKeyWords:
                allKeyWords.append(totalWords[i][j])
      
    allKeyWords.remove('class'); allKeyWords.remove('neweggcom')
    allKeyWords.remove('best'); allKeyWords.remove('buy')
    time1 = time.time()
    print('Ended dataprocessing (', timer, '/',timerfin,')', time1-time0)
    timer+=1
##----------------------------------------------------




##-- Inputmatrix    
    inputMatrix = np.zeros((len(allKeyWords), len(x_train2)))
    for k in range(len(allKeyWords)):
        for i in range(len(x_train2)):
            if allKeyWords[k] in textfunc2(x_train2['title'].iloc[i]):
                    inputMatrix[k,i] = 1
    time2 = time.time()
    print('Ended inputmatrix computation (', timer, '/',timerfin,')', time2-time0)
    timer+=1
##----------------------------------------------------




   
##-- Minhashing
    #Random number generator function with integer as input
    def randomNumberGenerator(value):
        #create list to store random values such that random values are all created once when function is called
        randomNumbers = []
        indexValue = random.randint(0, len(allKeyWords) - 1)
        randomNumbers.append(indexValue)

        while value > 0:
            while indexValue in randomNumbers:
                indexValue = random.randint(0, len(allKeyWords) - 1)
            randomNumbers.append(indexValue)
            value=value-1
        return randomNumbers

    numMinHash = hashNumber #(Input) # about half of number of keywords
    p = 1543

    
    randomIntA = randomNumberGenerator(numMinHash)
    randomIntB = randomNumberGenerator(numMinHash)
    time3 = time.time()
    print('Ended minhashing computation (', timer, '/',timerfin,')', time3-time0)
    timer+=1
##----------------------------------------------------




##-- SignatureMatrix
    signatureMatrix = []
    for i in range(len(x_train2)): #first loop over all products
        signature = []
        for j in range(0, numMinHash): #Do this 250 times
            minHashValue = 2**32 - 1
            for k in range(len(allKeyWords)): #then loop over all keywords
                if inputMatrix[k,i] == 1:
                    hashValue = (randomIntA[j] + k * randomIntB[j]) % p
                    if hashValue < minHashValue:                    
                        minHashValue = hashValue 

            signature.append(minHashValue)     
        signatureMatrix.append(signature)
    time4 = time.time()
    print('Ended signaturematrix computation (', timer, '/',timerfin,')', time4-time0)
    timer+=1
    signatureMatrixFin = np.transpose(pd.DataFrame(signatureMatrix))
##----------------------------------------------------




    

##--Splitting signaturematrix in corresponding rows and bands    
    z = np.arange(0,numMinHash,r)
    bandsDict = {}
    for i in range(len((z))-1):
        bandsDict["band{0}".format(i)]=signatureMatrixFin[z[i]:z[i+1]][:]
    bandsDict["band{0}".format(b-1)]=signatureMatrixFin[z[b-1]:][:]    
    time5 = time.time()
    print('Ended splitting signaturematrix into bands (', timer, '/',timerfin,')', time5-time0)
    timer+=1
##----------------------------------------------------



    
    
##--Create hashvalues for columns of each band
    values = bandsDict.values()
    keys = bandsDict.keys() #keys = ['band0', 'band1', ... , 'bandb']


    hashValBand = {}; hashValBandS = {}; hashValBandDupl = {}; hashValBandNoDupl = {}
    for i in range(0,b):
        hashValBandS[i] = {}

        for j in range(len(x_train)):
            valueHash1 = bandsDict['band'+str(i)][:][j]
            valueHash2 = [str(int) for int in valueHash1]
            valueHash3 = ''.join(valueHash2)
            if valueHash3 not in hashValBandS[i].keys():
                hashValBandS[i][valueHash3] = [j]
            else:
                hashValBandS[i][valueHash3].append(j)

        hashValBandDupl[i] = {}
        hashValBandNoDupl[i] = {}

        for key in hashValBandS[i]:
            if len(hashValBandS[i][key]) > 1:
                  hashValBandDupl[i][key] = hashValBandS[i][key]
            else:
                hashValBandNoDupl[i][key] = hashValBandS[i][key]
    time6 = time.time()
    print('Ended creating hashvalues for bands (', timer, '/',timerfin,')', time6-time0)
    timer+=1
##----------------------------------------------------




##--Extract all candidate pairs of all bands for further comparison and renaming
    candidateBuck = {}
    noCandidateBuck = {}
    for i in range(0,b):
        candidateBuck['candidatesBuck%s' % i] = list(hashValBandDupl[i].values())
        noCandidateBuck['noCandidatesBuck%s' % i] = list(hashValBandNoDupl[i].values())
    pairsB = {}
    noPairsB = {}
    for i in range(0,b):   
        pairsB[i] = candidateBuck['candidatesBuck'+str(i)]
        noPairsB[i] = noCandidateBuck['noCandidatesBuck'+str(i)]
        
##-- Add all (non)candidate pairs to a single list
    allPairs = []
    allNoPairs = []
    for i in range(0,b):
        allPairs.append(pairsB[i])
        allNoPairs.append(noPairsB[i]) 
        
##-- Remove duplicate (non)candidate pairs 
    allPairsDef = []
    allNonPairsDef = []
    for i in range(b):
        for j in range(len(allPairs[i])):
            if allPairs[i][j] not in allPairsDef:
                allPairsDef.append(allPairs[i][j])

    for i in range(b):
        for j in range(len(allNoPairs[i])):
            if allNoPairs[i][j] not in allNonPairsDef:
                allNonPairsDef.append(allNoPairs[i][j])
    time7 = time.time()
    print('Ended removing (non)duplicate pairs from allPairsDef (', timer, '/',timerfin,')', time7-time0)
    timer+=1
##----------------------------------------------------

    
    
    
        
##-- To compare only pairs (two products), take care of the buckets with multiple items
##-- by extracting and removing, and later adding in pairs
    pairsTwoPlus = []
    for i in range(len(allPairsDef)):
        if(len(allPairsDef[i]) > 2):
            pairsTwoPlus.append(allPairsDef[i])

    
##-- Make new pairs from buckets with more than 2 items        
    pairsTwoPlusDoubles = []
    for item in range(len(pairsTwoPlus)):
        newComp = [(item1, item2) for idx, item1 in enumerate(pairsTwoPlus[item]) for item2 in pairsTwoPlus[item][idx + 1:]]
        for item1,item2 in newComp:
            if [item2, item1] not in pairsTwoPlusDoubles:
                    pairsTwoPlusDoubles.append([item1,item2])
    
    
##-- Remove buckets with more than 2 items
    for removeMoreTwo in range(len(pairsTwoPlus)):
        allPairsDef.remove(pairsTwoPlus[removeMoreTwo])

    
    for i in pairsTwoPlusDoubles:
        allPairsDef.append(i)
#    print('number of candidatepairs', len(allPairsDef))
#     print(allPairsDef)

    time8 = time.time()
    print('Ended computing all bucketpairs across bands (', timer, '/',timerfin,')', time8-time0)
    timer+=1
##----------------------------------------------------


##-- Find all duplicate products using their modelID

    dupID = y_data2.pivot_table(columns=['modelID'], aggfunc='size')
    # print(dupID)
    dupID = dupID[dupID > 1]
    dupID2 = pd.DataFrame(dupID)
#    print("Total amount of double products:", sum(dupID2[0])) #691
#    print("Total ammount of duplicate products:", len(dupID)) #329
#     Dn = sum(dupID2[0])
    Dn = 691





##-- Find all duplicate products using their modelID
    dupCountList = []
    realDupl=0
    for i in y_train2['modelID']:
        if i not in dupCountList:
            dupCountList.append(i)
        else:
            realDupl+=1

            
            
##-- Products in the same shop are
    TP = 0; FP = 0
#     print(allPairsDef)
    for i in (allPairsDef):
        if y_train.iloc[i[0]] == y_train.iloc[i[1]]: #modelIDs similar
#             print('modelIDs equal:', i)
#             if x_train2['shop'].iloc[i[0]] != x_train2['shop'].iloc[i[1]]: #not from the same shop
#                 print('shops not equal:', i)
            TP+=1

        else:
            FP +=1
            
    Df = TP
    Nc = len(allPairsDef)
    PQ = Df / Nc
    PC = Df / Dn
    print("Df", Df)
#     print("Df:", Df, "Dn:", Dn, "Nc:", Nc, "TP:", TP, "FP:", FP, "PQ:", PQ, "PC:", PC)
    F1star = (2 * PQ * PC) / (PQ + PC)
#     print("F1star:", F1star)
    time9 = time.time()
    print('Ended LSH performance (', timer, '/',timerfin,')', time9-time0)
    timer+=1   
##----------------------------------------------------






##-- Similarity measures
    def jaccard(prod1, prod2):
        intersec = np.logical_and(prod1,prod2)
        union = np.logical_or(prod1, prod2)
        similarVal = intersec.sum() / float(union.sum())
        return similarVal

    def cosine(prod1, prod2):
        value = 0; length1 = 0; length2 = 0
        for i in range(len(prod1)):
            value += prod1[i]*prod2[i]
            length1 += (prod1[i])**2
            length2 += (prod2[i])**2
        sim = value / (np.sqrt(length1) * np.sqrt(length2))
        return sim
    
#     print('total amount of unique pairs found by LSH', len(allPairsDef))
    jaccardNew = np.zeros((len(allPairsDef)))
    cosineNew = np.zeros((len(allPairsDef)))

    
    for i in range(len(allPairsDef)):
        jaccardNew[i] = jaccard(inputMatrix[:, allPairsDef[i][0]], inputMatrix[:, allPairsDef[i][1]])
        cosineNew[i] = cosine(inputMatrix[:, allPairsDef[i][0]], inputMatrix[:, allPairsDef[i][1]])

    combinedSim = np.zeros((len(allPairsDef)))
    for i in range(len(allPairsDef)):
        if jaccardNew[i] >= thresholdSim and cosineNew[i] >= thresholdSim:
            combinedSim[i] = 1

    dfound = 0
    
    for i in range(len(allPairsDef)):
#         if combinedSim[i] == 0:
#             print(allPairsDef[i], allPairsDef[i[0]])
            
#             print('not candidatepair', i, allPairsDef[i])#, y_train.iloc[i[0]], y_train.iloc[i[1]])
#             allPairsDef.remove(allPairsDef[i])
        if combinedSim[i] == 1:
            a = allPairsDef[i]
#             print(allPairsDef[i])
            if y_train2['modelID'].iloc[a[0]] == y_train2['modelID'].iloc[a[1]]:
#                 print('true duplicate', i, allPairsDef[i], a)
                dfound +=1

    DfSim = dfound #sum of all 1.0's such
    NcSim = len(combinedSim)
    DnSim = Dn #remains the same for each subset
    
    PQSim = DfSim / NcSim
    PCSim = DfSim / DnSim
    F1Sim = (2 * PQSim * PCSim) / (PQSim + PCSim)
#     print()
#     print('DfSim:', DfSim, 'NcSim:', NcSim, 'DnSim', DnSim)
#     print("F1Sim:", F1Sim)
    
    
    time10 = time.time()
    print('Ended computing jaccard and cosine similarities (', timer, '/',timerfin,')', time10-time0)
    timer+=1

            
#     print('jaccardNew', jaccardNew)
#     print('cosineNew', cosineNew)
#     print('combined', combinedSim)
    
    
##-- Classification using jaccard- & cosine similarities and threshold
#     print("AllkeyWords:", len(allKeyWords))
#     print('LSH: [t, b, r, F1*]', t, b, r, F1star)
#     print('Classification: [t, b, r, F1]', t, b, r, F1Sim)
    maxCom = 1624 * 1623 * 0.5
    fracOfCompLSH = Nc / maxCom
    fracOfComp = NcSim / maxCom
    perfLSH = [PQ, PC, F1star, t, fracOfComp]
    perfSim = [PQSim, PCSim, F1Sim, t, fracOfComp]
    
    print('perfLSH', perfLSH)
    print('perfSim', perfSim)
    return perfLSH, perfSim


        
 

In [301]:
##---- prepare plots
PCStar = [] 
PC = []
PQStar = []
PQ = []
F1Star = []
F1 = []
noemerStar = []
noemer = []

##1 Pair Completeness
pairs = [(2, 250), (10,50), (20,25), (25, 20), (50,10), (100, 5), (125, 4), (250, 2), (500,1)]


plt.plot(noemerStar, PCStar, label='LSH', '-.')
plt.plot(noemer, PC, label='Classification', '-')
plt.ylabel('Pair Completeness')
plt.xlabel('Fraction of comparisons')

##2 Pair Quality

##3 F1*

##4 F1


    


In [ ]:
print(pairs[0])
y = fulltransformation(x_data, y_data, 500, 2, 250, 0.90)
# 

In [307]:
print(pairs[1])
y = fulltransformation(x_data, y_data, 500, 10, 50, 0.85)

(10, 50)
Ended dataprocessing ( 1 / 12 ) 0.6985888481140137
Ended inputmatrix computation ( 2 / 12 ) 19.011507987976074
Ended minhashing computation ( 3 / 12 ) 19.01863193511963
Ended signaturematrix computation ( 4 / 12 ) 296.45837473869324
Ended splitting signaturematrix into bands ( 5 / 12 ) 296.6094419956207
Ended creating hashvalues for bands ( 6 / 12 ) 297.49194383621216
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 297.5903408527374
Ended computing all bucketpairs across bands ( 8 / 12 ) 297.59077501296997
Df 399
Ended LSH performance ( 9 / 12 ) 297.60833191871643
Ended computing jaccard and cosine similarities ( 10 / 12 ) 298.14629769325256
perfLSH [0.9172413793103448, 0.5774240231548481, 0.7087033747779752, 0.954992586021436, 0.0003300765777660417]
perfSim [0.9172413793103448, 0.5774240231548481, 0.7087033747779752, 0.954992586021436, 0.0003300765777660417]


In [308]:
print(pairs[2])
y = fulltransformation(x_data, y_data, 500, 20, 25, 0.90)

(20, 25)
Ended dataprocessing ( 1 / 12 ) 0.6899659633636475
Ended inputmatrix computation ( 2 / 12 ) 19.27808713912964
Ended minhashing computation ( 3 / 12 ) 19.285283088684082
Ended signaturematrix computation ( 4 / 12 ) 293.96092677116394
Ended splitting signaturematrix into bands ( 5 / 12 ) 294.10724997520447
Ended creating hashvalues for bands ( 6 / 12 ) 295.6848440170288
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 295.8774299621582
Ended computing all bucketpairs across bands ( 8 / 12 ) 295.8778829574585
Df 404
Ended LSH performance ( 9 / 12 ) 295.8950581550598
Ended computing jaccard and cosine similarities ( 10 / 12 ) 296.4807300567627
perfLSH [0.8595744680851064, 0.5846599131693199, 0.6959517657192076, 0.8870718549931568, 0.0003566344633334244]
perfSim [0.8595744680851064, 0.5846599131693199, 0.6959517657192076, 0.8870718549931568, 0.0003566344633334244]


In [309]:
print(pairs[3])
y = fulltransformation(x_data, y_data, 500, 25, 20, 0.90)

(25, 20)
Ended dataprocessing ( 1 / 12 ) 0.6834070682525635
Ended inputmatrix computation ( 2 / 12 ) 19.10258388519287
Ended minhashing computation ( 3 / 12 ) 19.109740018844604
Ended signaturematrix computation ( 4 / 12 ) 296.17793583869934
Ended splitting signaturematrix into bands ( 5 / 12 ) 296.3223478794098
Ended creating hashvalues for bands ( 6 / 12 ) 298.2917559146881
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 298.5341899394989
Ended computing all bucketpairs across bands ( 8 / 12 ) 298.5350058078766
Df 425
Ended LSH performance ( 9 / 12 ) 298.55331087112427
Ended computing jaccard and cosine similarities ( 10 / 12 ) 299.2655611038208
perfLSH [0.7417102966841187, 0.6150506512301013, 0.6724683544303798, 0.8513399225207846, 0.0004347905265745791]
perfSim [0.7417102966841187, 0.6150506512301013, 0.6724683544303798, 0.8513399225207846, 0.0004347905265745791]


In [310]:
print(pairs[4])
y = fulltransformation(x_data, y_data, 500, 50, 10, 0.90)

(50, 10)
Ended dataprocessing ( 1 / 12 ) 0.67352294921875
Ended inputmatrix computation ( 2 / 12 ) 19.020033836364746
Ended minhashing computation ( 3 / 12 ) 19.0271737575531
Ended signaturematrix computation ( 4 / 12 ) 294.00558400154114
Ended splitting signaturematrix into bands ( 5 / 12 ) 294.1525218486786
Ended creating hashvalues for bands ( 6 / 12 ) 297.8049578666687
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 298.27912187576294
Ended computing all bucketpairs across bands ( 8 / 12 ) 298.5269069671631
Df 1120
Ended LSH performance ( 9 / 12 ) 298.5756437778473
Ended computing jaccard and cosine similarities ( 10 / 12 ) 305.3609137535095
perfLSH [0.2024584237165582, 1.6208393632416787, 0.35995500562429694, 0.6762433378062414, 0.004197663513107455]
perfSim [0.2024584237165582, 1.6208393632416787, 0.35995500562429694, 0.6762433378062414, 0.004197663513107455]


In [311]:
print(pairs[5])
y = fulltransformation(x_data, y_data, 500, 100, 5, 0.60)


(100, 5)
Ended dataprocessing ( 1 / 12 ) 0.6856448650360107
Ended inputmatrix computation ( 2 / 12 ) 19.102665901184082
Ended minhashing computation ( 3 / 12 ) 19.109761714935303
Ended signaturematrix computation ( 4 / 12 ) 294.6547887325287
Ended splitting signaturematrix into bands ( 5 / 12 ) 294.8058018684387
Ended creating hashvalues for bands ( 6 / 12 ) 301.9365727901459
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 303.054566860199
Ended computing all bucketpairs across bands ( 8 / 12 ) 515.9581429958344
Df 8176
Ended LSH performance ( 9 / 12 ) 516.8610608577728
Ended computing jaccard and cosine similarities ( 10 / 12 ) 690.8765640258789
perfLSH [0.05744437184270247, 11.832127351664255, 0.11433365962802405, 0.39810717055349726, 0.10799877985485735]
perfSim [0.05744437184270247, 11.832127351664255, 0.11433365962802405, 0.39810717055349726, 0.10799877985485735]


In [312]:
print(pairs[6])
y = fulltransformation(x_data, y_data, 500, 125, 4, 0.90)

(125, 4)
Ended dataprocessing ( 1 / 12 ) 0.6865499019622803
Ended inputmatrix computation ( 2 / 12 ) 19.072349786758423
Ended minhashing computation ( 3 / 12 ) 19.07956075668335
Ended signaturematrix computation ( 4 / 12 ) 293.9078288078308
Ended splitting signaturematrix into bands ( 5 / 12 ) 294.05485486984253
Ended creating hashvalues for bands ( 6 / 12 ) 302.8725817203522
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 304.5104658603668
Ended computing all bucketpairs across bands ( 8 / 12 ) 1779.2567510604858
Df 14676
Ended LSH performance ( 9 / 12 ) 1781.616888999939
Ended computing jaccard and cosine similarities ( 10 / 12 ) 2234.5529668331146
perfLSH [0.03978551174100922, 21.238784370477568, 0.079422245913483, 0.2990697562442441, 0.2799034203521424]
perfSim [0.03978551174100922, 21.238784370477568, 0.079422245913483, 0.2990697562442441, 0.2799034203521424]


In [313]:
print(pairs[7])
y = fulltransformation(x_data, y_data, 500, 250, 2, 0.90)

(250, 2)
Ended dataprocessing ( 1 / 12 ) 0.6706879138946533
Ended inputmatrix computation ( 2 / 12 ) 19.31565284729004
Ended minhashing computation ( 3 / 12 ) 19.32284688949585
Ended signaturematrix computation ( 4 / 12 ) 296.3901288509369
Ended splitting signaturematrix into bands ( 5 / 12 ) 296.544460773468
Ended creating hashvalues for bands ( 6 / 12 ) 313.83451199531555
Ended removing (non)duplicate pairs from allPairsDef ( 7 / 12 ) 316.5190668106079


KeyboardInterrupt: 

In [ ]:
print(pairs[8])
y = fulltransformation(x_data, y_data, 500, 500, 1, 0.90)

In [258]:
#combinations as input for fulltransformations
thresholdPairs = [(10, 50), (20, 25), (25, 20), (50, 10), (100, 5), (125, 4), (250, 2), (500, 1)]
#[(1, 400), (2, 200), (4, 100), (5, 80), (8, 50), (10,40), (20, 20), (40,10)] #400
maxComp = 1624 * 1623 * 0.5
#(b,r)
#(1,400) -> 1
#(8, 50) ->0.96
#(20,20) ->0.86
#(40,10) -> 0.69
#(100,4) -> 0.316
#(200,2) -> 0.07
#(400,1) -> 0.00
graphVal = [(1, 400), (2, 200), (4, 100), (5, 80), (8, 50), (10,40), (20, 20), (40,10)]
# for i in range(2)
y = fulltransformation(x_data, y_data, 400, 20, 20, 0.80)


SyntaxError: invalid syntax (<ipython-input-258-3353b8740cf0>, line 13)

In [ ]:
##---- Creating plots of r&b pairs
# b = first value
# r = second value

# thresholdPairs = [(1, 250), (2, 125), (4, 75), (5, 50), (10, 25), (25, 10)] #250
thresholdPairs = [(1, 400), (2, 200), (4, 100), (5, 80), (8, 50), (10,40), (20, 20), (40,10)] #400
# thresholdPairs = [(2, 250), (4, 125), (5, 100), (10, 50), (20, 25), (100, 5)] #500
# thresholdPairs = [(3, 250), (6, 125), (10, 75), (15, 50), (25, 30), (50, 15)] #750

print(thresholdPairs[0])
print(thresholdPairs[0][0]) #b of first pair
c = np.linspace(0.1,1,100)
col = ["red", "blue", "green", "black", "orange", "purple", "grey", "brown"]
# lab = ["r=3, b=10", "r=6, b=20", "r=5, b=50"]
threshold = [0,0,0,0,0,0,0,0]

def function(A, col):
    plt.plot(c, 1 - (1-c**A[1])**A[0], color=col, marker='o', label=['(b,r)=',A])
    plt.ylabel("S-curve"); plt.xlabel("s-value")
    plt.axhline(0.5)
    plt.title("S-curve for values of (r,b)")
    
for i in range(len(thresholdPairs)):
    function(thresholdPairs[i], col[i])
    threshold[i] = (1/thresholdPairs[i][0])**(1/thresholdPairs[i][1]) #approximations
    print("The threshold for r=",  thresholdPairs[i][1], "and b=", thresholdPairs[i][0], "is:", threshold[i])
#     plt.axvline(threshold[i], color=col[i], linestyle="--")
    plt.legend()    
